In [12]:
import pygame, sys, random, time #載入套件
from enum import Enum #列舉
from pygame.locals import * #載入套件
#玩家 
class Role(Enum):
    PLAYER = 'player'
    COMPUTER = 'computer'
#棋子顏色
class Side(Enum):
    BLACK = 'black'
    WHITE = 'white'
#模式
class Mode(Enum):
    FIRST = '先手'
    SECOND = '後攻'
    AI = 'AI_Battle'
    
#演算法_1
class Algorithm_1(object):
    #建構子
    def __init__(self,pUserTitle,pPossibleMoves,pBoard):
        self.mainboard = pBoard #複製後的棋盤位置
        self.possibleMoves = pPossibleMoves #合法位置
        self.userTitle = pUserTitle #執行玩家
        self.position = self.__main__()
    #取得位置
    def getPosition(self):
        return self.position
    # 是否在角上
    def isOnCorner(self,x, y):
        return (x == 0 and y == 0) or (x == 7 and y == 0) or (x == 0 and y == 7) or (x == 7 and y == 7)
    # 獲取棋盤上雙方的棋子數
    def getScoreOfBoard(self,board):
        xscore = 0
        oscore = 0
        for x in range(8):
            for y in range(8):
                if board[x][y] == 1:
                    xscore += 1
                if board[x][y] == 0:
                    oscore += 1
        return {1: xscore, 0: oscore}
    # 是否為合法走法
    def isValidMove(self,board, tile, xstart, ystart):
        # 檢查該位置是否出界或已有棋子
        if not self.isOnBoard(xstart, ystart) or board[xstart][ystart] is not None:
            return False

        # 臨時將tile放到指定的位置
        board[xstart][ystart] = tile

        if tile == 1:
            otherTile = 0
        else:
            otherTile = 1

        # 要被翻轉的棋子
        tilesToFlip = []
        for xdirection, ydirection in [[0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1]]:
            x, y = xstart, ystart
            x += xdirection
            y += ydirection
            # 前進方向第一格是 合法範圍 且 是對方的棋子
            if self.isOnBoard(x, y) and board[x][y] == otherTile:
                x += xdirection
                y += ydirection
                if not self.isOnBoard(x, y):
                    continue
                # 一直走到出界或是不是對方棋子
                while board[x][y] == otherTile:
                    x += xdirection
                    y += ydirection
                    if not self.isOnBoard(x, y):
                        break
                # 出界了，則没有棋子要翻轉
                if not self.isOnBoard(x, y):
                    continue
                # 是自己的棋子
                if board[x][y] == tile:
                    while True:
                        x -= xdirection
                        y -= ydirection
                        # 回到了起點則结束
                        if x == xstart and y == ystart:
                            break
                        # 需要翻轉的棋子
                        tilesToFlip.append([x, y])

        # 將前面臨時放上的棋子去掉，即還原棋盤
        board[xstart][ystart] = None  # restore the empty space

        # 没有要被翻轉的棋子，則走法非法
        if len(tilesToFlip) == 0:
            return False

        return tilesToFlip
    
    # 是否出界
    def isOnBoard(self,x, y):
        return 0 <= x <= 7 and 0 <= y <= 7
    # 將一個tile棋子放到(xstart, ystart)
    def makeMove(self,board, tile, xstart, ystart):
        tilesToFlip = self.isValidMove(board, tile, xstart, ystart)

        if tilesToFlip == False:
            return False

        board[xstart][ystart] = tile
        for x, y in tilesToFlip:
            board[x][y] = tile
        return True

    #主程式
    def __main__(self):
        # 打亂順序
        random.shuffle(self.possibleMoves)
        # [x, y]在角上，則優先走
        for x, y in self.possibleMoves:
            if self.isOnCorner(x, y):
                return [x, y]
        #預設最低分數
        bestScore = -1
        for x, y in self.possibleMoves:
            dupeBoard = self.mainboard
            self.makeMove(dupeBoard,self.userTitle, x, y)
            # 按照分數選擇走法，優先選擇翻轉後分數最多的走法
            score = self.getScoreOfBoard(dupeBoard)[self.userTitle]
            if score > bestScore:
                bestMove = [x, y]
                bestScore = score
        return bestMove


In [13]:
#演算法工廠
class Algorithm_Factory(object):
    #建構子
    def __init__(self,pFunc,pUserTitle,pPossibleMoves,pBoard):
        self.position = None
        if pUserTitle == Side.BLACK:
            pUserTitle = 1
        else:
            pUserTitle = 0
        pBoard = self.getBoardArray(pBoard)
        if pFunc == '1':
            self.position = Algorithm_1(pUserTitle,pPossibleMoves,pBoard).getPosition()
        else:
            self.position = Algorithm_1(pUserTitle,pPossibleMoves,pBoard).getPosition()
    #取得位置
    def getPosition(self):
        return self.position
    #取得棋盤array陣列 0:白棋 1:黑棋
    def getBoardArray(self,pBoard):
        for x in range(8):
            for y in range(8):
                if not pBoard[x][y] == None: 
                    if pBoard[x][y] == Side.BLACK:
                        pBoard[x][y] = 1
                    else:
                        pBoard[x][y] = 0
        return pBoard
                        
        


In [17]:
setFunc = ['1','1']
#Reversi物件
class Reversi(object):
    #建構子
    def __init__(self):
        #畫面設定
        self.WHITE = (255, 255, 255)#白色字體
        self.BLUE = (0, 0, 255)#藍色背景
        self.CELLWIDTH = 50;self.CELLHEIGHT = 50;self.PIECEWIDTH = 40;self.PIECEHEIGHT = 40
        self.BOARDX = 0;self.BOARDY = 0
        self.FPS = 1000
        self.useAIBattle = False #AI自鬥 Model
        # load img
        self.boardImage = pygame.image.load('./img/board.png')
        self.boardRect = self.boardImage.get_rect()
        self.blackImage = pygame.image.load('./img/black.png')
        self.blackRect = self.blackImage.get_rect()
        self.whiteImage = pygame.image.load('./img/white.png')
        self.whiteRect = self.whiteImage.get_rect()
        #預載物件與設定
        self.turn = None #輪流顯示幕前的玩家
        self.playerTile = None #玩家顏色
        self.computerTile = None #電腦顏色
        self.mode = None #遊戲模式
        #進入主程式
        self.__main__()
    
    # 棋盤
    def getNewBoard(sef):
        board = []
        for i in range(8):
            board.append([None] * 8)
        return board
    
    # 重置棋盤
    def resetBoard(self):
        self.mainBoard[3][3] = Side.BLACK
        self.mainBoard[3][4] = Side.WHITE
        self.mainBoard[4][3] = Side.WHITE
        self.mainBoard[4][4] = Side.BLACK
    
    # 離開
    def terminate(self):
        pygame.quit()
        sys.exit()
    
    #放置棋子
    def drawBoard(self):
        self.windowSurface.blit(self.boardImage, self.boardRect, self.boardRect)
        for x in range(0, 8):
            for y in range(0, 8):
                rectDst = pygame.Rect(self.BOARDX + x * self.CELLWIDTH + 5, self.BOARDY + y * self.CELLHEIGHT + 5, self.PIECEWIDTH, self.PIECEHEIGHT)
                if self.mainBoard[x][y] == Side.BLACK:
                    self.windowSurface.blit(self.blackImage, rectDst, self.blackRect)
                elif self.mainBoard[x][y] == Side.WHITE:
                    self.windowSurface.blit(self.whiteImage, rectDst, self.whiteRect)
    #設定按鈕
    def setBtnMode(self, pBtnList,pCenterList,pModeList):
        yAxis = int(self.boardRect.width / 2) + 40
        xAxis = int(400 / 2)
        btnList = []
        for i in range(len(pBtnList)):
            pBIGFONT = self.BIGFONT.render(pBtnList[i], True, self.WHITE, self.BLUE)
            pBtn = [pBIGFONT,pBIGFONT.get_rect(),pModeList[i]]
            pBtn[1].center = (xAxis + pCenterList[i], yAxis)
            btnList.append(pBtn)
        return btnList
    #選擇模式
    def setMode(self):
        pBtnStr =['black','white','AIBattle']
        pBtnPotion = [-120,0,120]
        pBtnMode = [Mode.FIRST,Mode.SECOND,Mode.AI]
        self.modeBtn = self.setBtnMode(pBtnStr,pBtnPotion,pBtnMode)
        while True:
            for event in pygame.event.get():
                #取得事件
                if event.type == QUIT:
                    self.terminate()
                if event.type == MOUSEBUTTONUP:
                    x, y = event.pos
                    for i in range(len(self.modeBtn)):
                        if self.modeBtn[i][1].collidepoint((x, y)):
                            return self.modeBtn[i][2]
            for i in range(len(self.modeBtn)):
                self.windowSurface.blit(self.modeBtn[i][0],self.modeBtn[i][1])
            pygame.display.update()
            self.mainClock.tick(self.FPS)
    # 遊戲是否結束
    def isGameOver(self):
        for x in range(8):
            for y in range(8):
                if self.mainBoard[x][y] is None:
                    return False
        if len(self.getValidMoves(Side.WHITE)) != 0 or len(self.getValidMoves(Side.BLACK)) != 0:
            return False
        return True
    # 獲取可落子的位置
    def getValidMoves(self, tile):
        validMoves = []

        for x in range(8):
            for y in range(8):
                if self.isValidMove(tile,self.getBoardCopy() , x, y):
                    validMoves.append([x, y])
        return validMoves
    
    
    # 是否為合法走法
    def isValidMove(self , tile,pBoard, xstart, ystart):
        # 檢查該位置是否出界或已有棋子
        if not self.isOnBoard(xstart, ystart) or pBoard[xstart][ystart] is not None:
            return False

        # 臨時將tile放到指定的位置
        pBoard[xstart][ystart] = tile

        if tile == Side.BLACK:
            otherTile = Side.WHITE
        else:
            otherTile = Side.BLACK

        # 要被翻轉的棋子
        tilesToFlip = []
        for xdirection, ydirection in [[0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1]]:
            x, y = xstart, ystart
            x += xdirection
            y += ydirection
            # 前進方向第一格是 合法範圍 且 是對方的棋子
            if self.isOnBoard(x, y) and pBoard[x][y] == otherTile:
                x += xdirection
                y += ydirection
                if not self.isOnBoard(x, y):
                    continue
                # 一直走到出界或是不是對方棋子
                while pBoard[x][y] == otherTile:
                    x += xdirection
                    y += ydirection
                    if not self.isOnBoard(x, y):
                        break
                # 出界了，則没有棋子要翻轉
                if not self.isOnBoard(x, y):
                    continue
                # 是自己的棋子
                if pBoard[x][y] == tile:
                    while True:
                        x -= xdirection
                        y -= ydirection
                        # 回到了起點則结束
                        if x == xstart and y == ystart:
                            break
                        # 需要翻轉的棋子
                        tilesToFlip.append([x, y])

        # 將前面臨時放上的棋子去掉，即還原棋盤
        pBoard[xstart][ystart] = None  # restore the empty space
        # 没有要被翻轉的棋子，則走法非法
        if len(tilesToFlip) == 0:
            return False   
        return tilesToFlip
    
    # 是否出界
    def isOnBoard(self,x, y):
        return 0 <= x <= 7 and 0 <= y <= 7

    # 獲取棋盤上雙方的棋子數
    def getScoreOfBoard(self):
        xscore = 0
        oscore = 0
        for x in range(8):
            for y in range(8):
                if self.mainBoard[x][y] == Side.BLACK:
                    xscore += 1
                if self.mainBoard[x][y] == Side.WHITE:
                    oscore += 1
        return {Side.BLACK: xscore, Side.WHITE: oscore}
    #結束訊息
    def getEndMsg(self):
        score = self.getScoreOfBoard(self)
        return 'Game Over Score '+ str(score[self.playerTile]) + ":" + str(score[self.computerTile])
    # 複製棋盤
    def getBoardCopy(self):
        dupeBoard = self.getNewBoard()

        for x in range(8):
            for y in range(8):
                dupeBoard[x][y] = self.mainBoard[x][y]

        return dupeBoard
    # 將一個tile棋子放到(xstart, ystart)
    def makeMove(self, tile, xstart, ystart):
        tilesToFlip = self.isValidMove( tile, self.getBoardCopy(), xstart, ystart)
        if tilesToFlip == False:
            return False

        self.mainBoard[xstart][ystart] = tile
        for x, y in tilesToFlip:
            self.mainBoard[x][y] = tile
        return True
    #主程式
    def __main__(self):
        # 初始化
        pygame.init()
        self.mainClock = pygame.time.Clock()
        self.basicFont = pygame.font.SysFont(None, 48)
        self.FONT = pygame.font.Font(None, 16)
        self.BIGFONT = pygame.font.Font(None, 32)

        self.mainBoard = self.getNewBoard()
        self.resetBoard()

        # 設置窗口
        self.windowSurface = pygame.display.set_mode((self.boardRect.width, self.boardRect.height))
        pygame.display.set_caption('Reversi')
        #判斷模式
        sel = self.setMode()
        if sel == Mode.AI:
            self.useAIBattle = True
            self.turn = Role.PLAYER
            self.playerTile = Side.BLACK
            self.computerTile = Side.WHITE
        elif sel == Mode.FIRST:
            self.turn = Role.PLAYER
            self.playerTile = Side.BLACK
            self.computerTile = Side.WHITE
        elif sel == Mode.SECOND:
            self.turn = Role.COMPUTER
            self.playerTile = Side.WHITE
            self.computerTile = Side.BLACK
        # 遊戲主循環
        while True:
            for event in pygame.event.get():
                if event.type == QUIT:
                    self.terminate()
                if self.isGameOver():
                    self.drawBoard()
                    text = basicFont.render(self.getEndMsg(), True, BLACK, BLUE)
                    textRect = text.get_rect()
                    textRect.centerx = windowSurface.get_rect().centerx
                    textRect.centery = windowSurface.get_rect().centery
                    windowSurface.blit(text, textRect)
                elif self.turn == Role.PLAYER:
                    #取得玩家下的位置
                    if self.useAIBattle:
                        # 獲取所有合法走法
                        possibleMoves = self.getValidMoves( self.playerTile)
                        dupeBoard =  self.getBoardCopy()
                        col, row = Algorithm_Factory(setFunc[0],playerTile,possibleMoves,dupeBoard).getPosition()
                        if self.makeMove(  self.playerTile, col, row):
                            if len(self.getValidMoves(  self.computerTile)) != 0:
                                self.drawBoard()
                                self.turn = Role.COMPUTER
                                self.drawBoard()
                    else:
                        if event.type == MOUSEBUTTONUP:
                            x, y = event.pos
                            col = int((x - self.BOARDX) / self.CELLWIDTH)
                            row = int((y - self.BOARDY) / self.CELLHEIGHT)
                            if self.makeMove( self.playerTile, col, row):
                                if len(self.getValidMoves(  self.computerTile)) != 0:
                                    self.drawBoard()
                                    self.turn = Role.COMPUTER
                                    self.drawBoard()
                elif self.turn == Role.COMPUTER:
                    #取得電腦下的位置
                    #演算法
                    possibleMoves = self.getValidMoves( self.computerTile)
                    dupeBoard =  self.getBoardCopy()
                    col, row = Algorithm_Factory(setFunc[1],self.computerTile,possibleMoves,dupeBoard).getPosition()
                    if self.makeMove( self.computerTile, col, row):
                        if len(self.getValidMoves( self.playerTile)) != 0:
                            self.drawBoard()
                            self.turn = Role.PLAYER
                            self.drawBoard()
                self.drawBoard()
            pygame.display.update()
            self.mainClock.tick(self.FPS)

            
reversi = Reversi()

Side.BLACK 3 5
可以下的位置:
[2, 3]
可以下的位置:
[2, 5]
可以下的位置:
[4, 5]
[[2, 3], [2, 5], [4, 5]]
222
111
可以下的位置:
[2, 3]
可以下的位置:
[2, 5]
可以下的位置:
[4, 5]
[[2, 3], [2, 5], [4, 5]]
[[2, 3], [2, 5], [4, 5]]
Side.WHITE 4 5
可以下的位置:
[5, 2]
可以下的位置:
[5, 3]
可以下的位置:
[5, 4]
可以下的位置:
[5, 5]
可以下的位置:
[5, 6]
[[5, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
5
可以下的位置:
[5, 2]
可以下的位置:
[5, 3]
可以下的位置:
[5, 4]
可以下的位置:
[5, 5]
可以下的位置:
[5, 6]
[[5, 2], [5, 3], [5, 4], [5, 5], [5, 6]]
Side.BLACK 5 4
可以下的位置:
[2, 3]
可以下的位置:
[2, 5]
可以下的位置:
[6, 3]
可以下的位置:
[6, 5]
[[2, 3], [2, 5], [6, 3], [6, 5]]
222
111
可以下的位置:
[2, 3]
可以下的位置:
[2, 5]
可以下的位置:
[6, 3]
可以下的位置:
[6, 5]
[[2, 3], [2, 5], [6, 3], [6, 5]]
[[2, 3], [2, 5], [6, 3], [6, 5]]
Side.WHITE 2 3
可以下的位置:
[2, 2]
可以下的位置:
[2, 4]
可以下的位置:
[3, 2]
可以下的位置:
[3, 6]
可以下的位置:
[4, 2]
可以下的位置:
[4, 6]
可以下的位置:
[5, 5]
[[2, 2], [2, 4], [3, 2], [3, 6], [4, 2], [4, 6], [5, 5]]
7
可以下的位置:
[2, 2]
可以下的位置:
[2, 4]
可以下的位置:
[3, 2]
可以下的位置:
[3, 6]
可以下的位置:
[4, 2]
可以下的位置:
[4, 6]
可以下的位置:
[5, 5]
[[2, 2], [2, 4], [3, 2], [3, 6], [4, 

SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#演算法物件